# Bias-Korrektur

Die ReKlies-Daten liegen ab 2006 vor. Zwischen 2006 und und 2023 liegen also sowohl Reanalyse- als auch Klimamodelldaten vor. Diesen Zeitraum kann man nutzen, um die Klimamodelldaten auf einen systematischen Fehler, einen so genannten Bias zu überprüfen. Sind die Werte im Mittel höher oder niedriger als die Reanalysedaten des gleichen Zeitraums, passt man die einzelne Werte um diesen Unterschied an, so dass sie das Klima des Untersuchungsgebiets besser abbilden.

In [4]:
# benötigte libraries importieren  
import pandas as pd
import os

In [5]:
# Dateien einlesen
historical = pd.read_csv("https://raw.githubusercontent.com/emorl/projektstudie-murgtal/main/1_daten/era5_tas_prec_discharge_merge.csv")
future = pd.read_csv("https://raw.githubusercontent.com/emorl/projektstudie-murgtal/main/1_daten/tas_pr_merge.csv")

In [6]:
# aus beiden Datensätzen neue Datensätze für 2006-2023 erstellen zum Vergleich

future['date'] = pd.to_datetime(future['date'])
future06_23 = future[future['date'] <= '2023-12-31']
future06_23 = future06_23.reset_index(drop=True)

historical['date'] = pd.to_datetime(future['date'])
historical06_23 = historical[historical['date'] <= '2023-12-31']
historical06_23 = historical06_23.reset_index(drop=True)

In [7]:
# Durchschnitte aller Parameter berechnen
future_means = future06_23.mean(numeric_only=True)
historical_means = historical06_23.mean(numeric_only=True)
print(future_means, historical_means)


pr_day_rcp26     3.834360
tas_day_rcp26    7.733006
pr_day_rcp85     4.160503
tas_day_rcp85    8.142320
dtype: float64 t2m_mean      8.513729
tp_sum        3.660127
flow_m3_s    16.166719
dtype: float64


Im Vergleichszeitraum 2006 bis 2023 ist die durchschnittliche Temperatur ca. 8,51°C und die durchschnittliche tägliche Niederschlagsmenge 3,66mm. <br>
Aus den Reanalysedaten geht für den Vergleichszeitraum 2006 bis 2023 eine durchschnittliche Temperatur von ca. 8,51°C und ein durchschnittlicher täglicher Niederschlag von ca. 3,66mm hervor. Das Klimamodell im Szenario RCP 2.6 hat im gleichen Zeitraum eine Durchschnittstemperatur von ca. 7,72°C und einen durchschnittlichen täglichen Niedershclag von ca. 3,83mm, im RCP 8.5 Szenario 8,24°C und 3,66°C. <br>
Das Klimamodell unterschätzt also den Niederschlag und überschätzt die Temperatur. <br>
Der Niederschlagsbias wird durch eine multiplikative Biaskorrektur korrigiert, folgend der Formel <br>
Niederschlag korrigiert = Wert Modell  x (Durchschnitt Historisch / Durchschnitt Modell) <br>
Der Temperaturbias wird durch eine additive Biaskorrektur ergänzt, da hier die Werte auch negativ werden können. Die Formel lautet <br>
Temperatur korrigiert = Temperatur Modell - (Durchschnitt Modell - Durchschnitt Historisch) 

In [8]:
# im Reanalysedatensatz heißen die spalten tas und pr (temperature air surface und precipitation), 
# im Modellsatz t2m_mean und tp_sum (temperature 2 meter (above ground) and total precipitation sum)
# mit mapping werden die beiden Spaltnenamen einander zugeordnet
mapping = {}
for col in future06_23.columns:
    if "pr" in col:
        mapping[col] = "tp_sum"        
    elif "tas" in col:
        mapping[col] = "t2m_mean"      

# neuer df für die korrgierten Werte nach Vorgabe des ersten
future06_23_corrected = future06_23.copy()

# alte Werte austauschen 
for model_col, hist_col in mapping.items():
    model_mean = future_means[model_col]
    hist_mean = historical_means[hist_col]
# Niederschlag
    if "pr" in model_col:
        # multiplikative Biaskorrektur 
        scale = hist_mean / model_mean
        future06_23_corrected[model_col] = future06_23[model_col] * scale
# Temperatur
    else:
        # additive Biaskorrektur
        future06_23_corrected[model_col] = (
            future06_23_corrected[model_col] - model_mean + hist_mean
        )


In [9]:
# überprüfen 
historical06_23.mean(), future06_23.mean(), future06_23_corrected.mean(),


(date         2014-12-31 12:00:00
 t2m_mean                8.513729
 tp_sum                  3.660127
 flow_m3_s              16.166719
 dtype: object,
 date             2014-12-31 12:00:00
 pr_day_rcp26                 3.83436
 tas_day_rcp26               7.733006
 pr_day_rcp85                4.160503
 tas_day_rcp85                8.14232
 dtype: object,
 date             2014-12-31 12:00:00
 pr_day_rcp26                3.660127
 tas_day_rcp26               8.513729
 pr_day_rcp85                3.660127
 tas_day_rcp85               8.513729
 dtype: object)

Die Durchschnitte zwischen historical und future sind nach der Biaskorrektur gleich, die Biaskorrektur war erfolgreich.
Nach dem gleichen Verfahren werden alle Modelldaten korrigiert werden.

In [10]:
future_bias_corrected = future.copy()

for model_col, hist_col in mapping.items():

    if "pr" in model_col:
        # multiplikative Biaskorrektur  
        scale = hist_mean / model_mean
        future_bias_corrected[model_col] = future[model_col] * scale

    else:
        # additive Biaskorrektur
        future_bias_corrected[model_col] = (
            future_bias_corrected[model_col] - model_mean + hist_mean
        )

In [ ]:
# korrigierten Datensatz lokal speichern
future_bias_corrected.to_csv("C:/Users/henry/Documents/A git/projektstudie-murgtal/1_daten/future_bias_corrected.csv", index=False)